In [16]:
# !pip install p_tqdm
# !pip install psycopg2

In [1]:
import psycopg2
from psycopg2.extensions import register_adapter
from psycopg2.extras import Json

register_adapter(dict, Json)

conn = psycopg2.connect("dbname=collegepdf user=postgres")

cur = conn.cursor()

sql_1 = "DROP TABLE IF EXISTS textract;"
sql_2 = r"""
CREATE TABLE textract (
  id SERIAL PRIMARY KEY,
  filepath VARCHAR(255) NOT NULL,
  filename VARCHAR(255) NOT NULL,
  college VARCHAR (255) NOT NULL,
  pdfminer TEXT,
  pdfminer_detailed JSON[],
  fonts JSON,
  pypdf2 TEXT,
  pypdf2_detailed JSON[]
);
""".replace('\n','')

for sql in [sql_1,sql_2]:
    cur.execute(sql)
    
sql_3 = """
INSERT INTO textract 
  (filepath, filename, college, pdfminer, pdfminer_detailed, pypdf2) 
  VALUES (%(filepath)s, %(filename)s, %(college)s, %(pdfminer)s, %(pdfminer_detailed)s::json[], %(pypdf2)s)
""".replace('\n','')

conn.commit()

In [2]:
import os

from tqdm.notebook import trange, tqdm
from p_tqdm import p_imap

from utils.pdf_processing import get_all_data

batch_1 = ['Arkansas', 'Bowdoin', 'Ohio State']
batch_2 = ['Colorado - Boulder', 'Minnesota Twin Cities', 'UCLA', 'U Conn']
limit_folders = batch_1 + batch_2
# limit_folders = []

files = [directory + '/' + file 
         for directory,_,files in list(os.walk('data')) 
         for file in files 
         if '.pdf' in file
         and (directory.split('/')[1] in limit_folders 
              if len(limit_folders) != 0
              else True)]

fn = lambda path: get_all_data(path)
# multiprocessing + tqdm
iterator = p_imap(fn, files)#, ['a', 'b', 'c'])

for result in iterator:
    try:
        cur.execute(sql_3, result)
        print(result['filename'])
    except Exception as e:
        print(f, e)

/home/marcin/.local/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='data/UCLA/UCLA_2001–03.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)
/home/marcin/.local/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='data/UCLA/UCLA_2007–08.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)
/home/marcin/.local/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='data/UCLA/UCLA_2009–10.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warning

UCLA_2017–18.pdf
UCLA_2009–10.pdf
UCLA_1997–99.pdf
UCLA_2007–08.pdf
UCLA_1995–97.pdf
UCLA_2013–14.pdf
UCLA_2001–03.pdf


UCLA_2008–09.pdf
UCLA_2010–11.pdf
UCLA_2016–17.pdf
UCLA_2011–12.pdf


UCLA_2019–20.pdf


UCLA_2018–19.pdf


...

UCLA_2014–15.pdf
UCLA_1991–92.pdf
UCLA_2015–16.pdf
UCLA_1993–94.pdf
UCLA_1999–2001.pdf
U Conn_2008-2009.pdf
U Conn_2010-2011.pdf
U Conn_2007-2008.pdf
U Conn_2015-2016.pdf
U Conn_2016-2017.pdf
U Conn_2014-2015.pdf
U Conn_2006-2007.pdf
U Conn_2017-2018.pdf
U Conn_1998-1999.pdf
U Conn_2005-2006.pdf
U Conn_2000-2001.pdf
U Conn_2018-2019.pdf
U Conn_2012-2013.pdf
U Conn_2002-2003.pdf
U Conn_2003-2004.pdf
U Conn_1999-2000.pdf
U Conn_2011-2012.pdf
U Conn_2009-2010.pdf
U Conn_2001-2002.pdf
U Conn_2004-2005.pdf


Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-7:
Process ForkPoolWorker-10:
Process ForkPoolWorker-3:
Process ForkPoolWorker-11:
Process ForkPoolWorker-4:
Process ForkPoolWorker-8:
Process ForkPoolWorker-6:
Process ForkPoolWorker-12:
Process ForkPoolWorker-9:
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/marcin/.local/lib/python3.8/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/home/marcin/.local/lib/python3.8/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/marcin/.local/lib/python3.8/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File 

  File "/home/marcin/.local/lib/python3.8/site-packages/pdfminer/pdfinterp.py", line 895, in process_page
    self.render_contents(page.resources, page.contents, ctm=ctm)
  File "/home/marcin/.local/lib/python3.8/site-packages/dill/_dill.py", line 454, in dump
    StockPickler.dump(self, obj)
  File "<ipython-input-2-9c778be63b04>", line 21, in <lambda>
    fn = lambda path: get_all_data(path)
  File "/home/marcin/.local/lib/python3.8/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/pickle.py", line 485, in dump
    self.save(obj)
  File "/usr/lib/python3.8/pickle.py", line 884, in save_tuple
    save(element)
  File "/home/marcin/.local/lib/python3.8/site-packages/pdfminer/pdfinterp.py", line 896, in process_page
    self.device.end_page(page)
  File "/home/marcin/Projects/upwork/Jacob/utils/pdf_processing.py", line 121, in get_all_data
    for page in doc:
  File "/home/marcin/.local/lib/python3.8/site-p

  File "/home/marcin/.local/lib/python3.8/site-packages/pdfminer/layout.py", line 795, in analyze
    textlines = list(self.group_objects(laparams, textobjs))
  File "/usr/lib/python3.8/pickle.py", line 558, in save
    f(self, obj)  # Call unbound method with explicit self
  File "/home/marcin/.local/lib/python3.8/site-packages/pdfminer/layout.py", line 795, in analyze
    textlines = list(self.group_objects(laparams, textobjs))
  File "/home/marcin/.local/lib/python3.8/site-packages/pdfminer/pdfinterp.py", line 802, in do_TJ
    self.device.render_string(self.textstate, seq, self.ncs,
  File "/home/marcin/.local/lib/python3.8/site-packages/pdfminer/pdfinterp.py", line 896, in process_page
    self.device.end_page(page)
  File "/home/marcin/Projects/upwork/Jacob/utils/pdf_processing.py", line 40, in pdfminer_to_text
    interpreter.process_page(page)
  File "/usr/lib/python3.8/pickle.py", line 884, in save_tuple
    save(element)
KeyboardInterrupt
  File "/home/marcin/.local/lib/pytho

  File "/usr/lib/python3.8/pickle.py", line 995, in _batch_setitems
    save(v)
  File "/home/marcin/.local/lib/python3.8/site-packages/pdfminer/pdfinterp.py", line 908, in render_contents
    self.execute(list_value(streams))
  File "/usr/lib/python3.8/pickle.py", line 995, in _batch_setitems
    save(v)
  File "/usr/lib/python3.8/pickle.py", line 558, in save
    f(self, obj)  # Call unbound method with explicit self
  File "/home/marcin/.local/lib/python3.8/site-packages/pdfminer/layout.py", line 680, in group_textlines
    neighbors = line.find_neighbors(plane, laparams.line_margin)
  File "/usr/lib/python3.8/pickle.py", line 558, in save
    f(self, obj)  # Call unbound method with explicit self
  File "/home/marcin/.local/lib/python3.8/site-packages/pdfminer/pdfinterp.py", line 933, in execute
    func(*args)
  File "/usr/lib/python3.8/pickle.py", line 929, in save_list
    self._batch_appends(obj)
  File "/home/marcin/.local/lib/python3.8/site-packages/pdfminer/layout.py", line 

In [3]:
conn.commit()

In [1]:
import psycopg2
conn = psycopg2.connect("dbname=collegepdf user=postgres")
cur = conn.cursor()
cur.execute("select filename from textract where college='UCLA'")
cur.fetchall()

[('UCLA_2017–18.pdf',),
 ('UCLA_2009–10.pdf',),
 ('UCLA_1997–99.pdf',),
 ('UCLA_2007–08.pdf',),
 ('UCLA_1995–97.pdf',),
 ('UCLA_2013–14.pdf',),
 ('UCLA_2001–03.pdf',),
 ('UCLA_2008–09.pdf',),
 ('UCLA_2010–11.pdf',),
 ('UCLA_2016–17.pdf',),
 ('UCLA_2011–12.pdf',),
 ('UCLA_2019–20.pdf',),
 ('UCLA_2018–19.pdf',),
 ('UCLA_1992–93.pdf',),
 ('UCLA_2003–05.pdf',),
 ('UCLA_2012–13.pdf',),
 ('UCLA_1994–95.pdf',),
 ('UCLA_2005–07.pdf',),
 ('UCLA_2014–15.pdf',),
 ('UCLA_1991–92.pdf',),
 ('UCLA_2015–16.pdf',),
 ('UCLA_1993–94.pdf',),
 ('UCLA_1999–2001.pdf',)]